In [2]:
import pandas as pd
import boto3
from io import BytesIO
from botocore.exceptions import ClientError

# -------------------------------------------------------------
# 1. MinIO-Verbindungsdaten & Bucket-Initialisierung
# -------------------------------------------------------------
MINIO_ENDPOINT   = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
BUCKET_NAME      = "batch-bucket"

# boto3-S3-Client für MinIO
s3 = boto3.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

# Bucket anlegen, falls nicht vorhanden
existing_buckets = [b["Name"] for b in s3.list_buckets().get("Buckets", [])]
if BUCKET_NAME not in existing_buckets:
    s3.create_bucket(Bucket=BUCKET_NAME)


# -------------------------------------------------------------
# 2. Funktion: pandas-DataFrame als Parquet-Byte-Stream nach MinIO hochladen (mit Vorab-Löschung)
# -------------------------------------------------------------
def upload_df_to_minio(
    df: pd.DataFrame,
    bucket: str,
    key: str,
    s3_client: boto3.client
) -> None:
    """
    Schreibt das gegebene pandas-DataFrame als Parquet in einen In-Memory-Buffer
    und lädt es anschließend per boto3 in den angegebenen MinIO-Bucket hoch.
    Falls schon ein Objekt unter demselben Key existiert, wird es zuerst gelöscht,
    um „resource deadlock avoided“ zu vermeiden.
    """
    # 2.1 Versuche, vorhandenes Objekt zu löschen (ignoriert 404-Fehler)
    try:
        s3_client.head_object(Bucket=bucket, Key=key)
        # Existiert → löschen
        s3_client.delete_object(Bucket=bucket, Key=key)
        print(f"Vorhandenes Objekt '{key}' wurde gelöscht, bevor neu geschrieben wurde.")
    except ClientError as e:
        code = e.response["Error"]["Code"]
        if code == "404":
            # Objekt existiert nicht → einfach weiter
            pass
        else:
            # Alle anderen Fehler weitergeben
            raise

    # 2.2 DataFrame in Parquet-Bytes schreiben
    buffer = BytesIO()
    df.to_parquet(buffer, engine="pyarrow", index=False)
    buffer.seek(0)

    # 2.3 Neues Parquet-Objekt hochladen
    try:
        s3_client.put_object(
            Bucket=bucket,
            Key=key,
            Body=buffer.read()
        )
        print(f"DataFrame erfolgreich als Parquet hochgeladen: s3://{bucket}/{key}")
    except ClientError as upload_err:
        print(f"Fehler beim Hochladen nach MinIO: {upload_err}")
        raise


# -------------------------------------------------------------
# 3. Wetter-Rohdaten einlesen und hochladen (Beispiel)
# -------------------------------------------------------------
weather_csv_path = "../input_data/weather_2023_2024.csv"
df_weather = pd.read_csv(weather_csv_path)

weather_key = "bronze/weather/raw_weather.parquet"
upload_df_to_minio(
    df=df_weather,
    bucket=BUCKET_NAME,
    key=weather_key,
    s3_client=s3
)


# -------------------------------------------------------------
# 4. Retail-Rohdaten einlesen und hochladen (Beispiel)
# -------------------------------------------------------------
retail_csv_path = "../input_data/retail_sales_2023_2024.csv"
df_retail = pd.read_csv(retail_csv_path)

retail_key = "bronze/retail/raw_retail.parquet"
upload_df_to_minio(
    df=df_retail,
    bucket=BUCKET_NAME,
    key=retail_key,
    s3_client=s3
)

Vorhandenes Objekt 'bronze/weather/raw_weather.parquet' wurde gelöscht, bevor neu geschrieben wurde.
DataFrame erfolgreich als Parquet hochgeladen: s3://batch-bucket/bronze/weather/raw_weather.parquet
Vorhandenes Objekt 'bronze/retail/raw_retail.parquet' wurde gelöscht, bevor neu geschrieben wurde.
DataFrame erfolgreich als Parquet hochgeladen: s3://batch-bucket/bronze/retail/raw_retail.parquet
